In [2]:
import torch
import torch.nn as nn
import torch_pruning as tp
import matplotlib.pyplot as plt
from torch import optim
import os
import numpy as np
import copy
import json
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from typing import List, Tuple

# Configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_BASE_NAME = "lstm_nasa"
print(f"Using device: {DEVICE}")

# NASA Dataset preprocessing functions from your provided code
column_names = ['unit_number', 'time_in_cycles'] + [f'op_setting_{i}' for i in range(1, 4)] + [f'sensor_{i}' for i in
                                                                                               range(1, 24)]


def load_dataframe(file_path: str) -> pd.DataFrame | None:
    """Loads a single CMaps data file."""
    try:
        df = pd.read_csv(file_path, sep=' ', header=None, names=column_names)
        # Drop the last two columns if they are all NaNs (often artifacts of space delimiter)
        df.dropna(axis=1, how='all', inplace=True)
        return df
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None


def clean_data(df: pd.DataFrame) -> list:
    """Identifies columns to remove based on low std dev."""
    if df is None:
        return []
    # Columns with std dev < 0.02 (potential candidates for removal)
    cols_to_check = [col for col in df.columns if 'sensor' in col or 'op_setting' in col]
    low_std_cols = [col for col in cols_to_check if df[col].std() < 0.02]
    print(f"Columns with std < 0.02 (potential removal): {low_std_cols}")
    return low_std_cols


def add_rul(df: pd.DataFrame) -> pd.DataFrame | None:
    """Calculates and adds the Remaining Useful Life (RUL) column."""
    if df is None:
        return None
    max_cycles = df.groupby('unit_number')['time_in_cycles'].max().reset_index()
    max_cycles.columns = ['unit_number', 'max_cycle']
    df = df.merge(max_cycles, on='unit_number', how='left')
    df['RUL'] = df['max_cycle'] - df['time_in_cycles']
    df.drop(columns=['max_cycle'], inplace=True)
    # Clip RUL (optional, common practice to limit max RUL)
    df['RUL'] = df['RUL'].clip(upper=125)
    return df


def normalize_data(df: pd.DataFrame,
                   columns_to_normalize: List[str], scaler: MinMaxScaler = None) -> Tuple[pd.DataFrame, MinMaxScaler] | \
                                                                                    Tuple[None, None]:
    """Normalizes specified columns using MinMaxScaler."""
    if df is None:
        return None, None
    data_to_scale = df[columns_to_normalize]
    if scaler is None:
        scaler = MinMaxScaler()
        df[columns_to_normalize] = scaler.fit_transform(data_to_scale)
    else:
        # Ensure only columns present in the scaler are transformed
        valid_cols = [col for col in columns_to_normalize if col in scaler.feature_names_in_]
        if len(valid_cols) < len(columns_to_normalize):
            print("Warning: Some columns not found in the provided scaler. Skipping them.")
        if valid_cols:
            df[valid_cols] = scaler.transform(df[valid_cols])
    return df, scaler


def prepare_cmapss_data(data_dir: str, train_file: str, test_file: str, test_rul_file: str) -> Tuple[
    pd.DataFrame, pd.DataFrame, pd.DataFrame, MinMaxScaler, List[str]]:
    """Loads, cleans, preprocesses train/test data and RUL."""
    print("--- Preparing Training Data ---")
    train_df = load_dataframe(os.path.join(data_dir, train_file))
    train_df = add_rul(train_df)

    print("\n--- Preparing Test Data ---")
    test_df = load_dataframe(os.path.join(data_dir, test_file))
    test_rul_df = pd.read_csv(os.path.join(data_dir, test_rul_file), header=None, names=['RUL'])

    # Clean Data - identify columns based on TRAINING data variance
    cols_to_remove = clean_data(train_df)
    feature_cols = [col for col in train_df.columns if
                    col not in ['unit_number', 'time_in_cycles', 'RUL'] + cols_to_remove]
    print(f"\nUsing Features: {feature_cols}")

    # Drop removed columns from both train and test
    train_df.drop(columns=cols_to_remove, inplace=True, errors='ignore')
    test_df.drop(columns=cols_to_remove, inplace=True, errors='ignore')

    # Normalize features based on TRAINING data
    print("\n--- Normalizing Data ---")
    train_df_norm, scaler = normalize_data(train_df.copy(), feature_cols, scaler=None)
    test_df_norm, _ = normalize_data(test_df.copy(), feature_cols, scaler=scaler)

    return train_df_norm, test_df_norm, test_rul_df, scaler, feature_cols


# Custom Dataset for NASA time series (LSTM version - no flattening)
class NASALSTMDataset(Dataset):
    def __init__(self, df: pd.DataFrame, feature_cols: List[str], window_size: int = 50,
                 stride: int = 1, is_test: bool = False, test_rul_df: pd.DataFrame = None):
        self.df = df
        self.feature_cols = feature_cols
        self.window_size = window_size
        self.stride = stride
        self.is_test = is_test
        self.test_rul_df = test_rul_df
        self.samples = []
        self.targets = []

        self._prepare_samples()

    def _prepare_samples(self):
        """Create windowed samples from the dataframe"""
        units = self.df['unit_number'].unique()

        for unit in units:
            unit_df = self.df[self.df['unit_number'] == unit].sort_values('time_in_cycles')

            # For test data, we only need the last window for each unit
            if self.is_test:
                if len(unit_df) >= self.window_size:
                    window_data = unit_df[self.feature_cols].iloc[-self.window_size:].values
                    self.samples.append(window_data)
                    # Get RUL from test_rul_df
                    if self.test_rul_df is not None:
                        self.targets.append(self.test_rul_df.iloc[unit - 1]['RUL'])
                else:
                    # Pad if necessary
                    window_data = unit_df[self.feature_cols].values
                    padded = np.zeros((self.window_size, len(self.feature_cols)))
                    padded[-len(window_data):] = window_data
                    self.samples.append(padded)
                    if self.test_rul_df is not None:
                        self.targets.append(self.test_rul_df.iloc[unit - 1]['RUL'])
            else:
                # For training data, create multiple windows
                for i in range(0, len(unit_df) - self.window_size + 1, self.stride):
                    window_data = unit_df[self.feature_cols].iloc[i:i + self.window_size].values
                    rul = unit_df['RUL'].iloc[i + self.window_size - 1]
                    self.samples.append(window_data)
                    self.targets.append(rul)

        self.samples = np.array(self.samples, dtype=np.float32)
        self.targets = np.array(self.targets, dtype=np.float32)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        # For LSTM, keep the sequence shape (window_size, num_features)
        sample = self.samples[idx]
        target = self.targets[idx]
        return torch.FloatTensor(sample), torch.FloatTensor([target])


# LSTM Model Definition
class NASALSTM(nn.Module):
    def __init__(self, input_size, hidden_size=100, num_layers=2, fc_hidden_sizes=[64, 32], dropout_rate=0.2):
        super(NASALSTM, self).__init__()

        # LSTM layers
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout_rate if num_layers > 1 else 0
        )

        # Fully connected layers after LSTM
        fc_layers = []
        prev_size = hidden_size

        for fc_hidden_size in fc_hidden_sizes:
            fc_layers.extend([
                nn.Linear(prev_size, fc_hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = fc_hidden_size

        # Output layer for regression
        fc_layers.append(nn.Linear(prev_size, 1))

        self.fc = nn.Sequential(*fc_layers)

    def forward(self, x):
        # LSTM forward pass
        # x shape: (batch_size, sequence_length, input_size)
        lstm_out, (hidden, cell) = self.lstm(x)

        # Use the last hidden state
        # lstm_out shape: (batch_size, sequence_length, hidden_size)
        # We take the last time step
        last_hidden = lstm_out[:, -1, :]

        # Pass through fully connected layers
        output = self.fc(last_hidden)

        return output


def get_data_loaders(data_dir='./data/NASA', batch_size=128, window_size=50, val_split=0.2, seed=42):
    """Load NASA C-MAPSS dataset with train/val/test splits"""
    print(f"Loading NASA C-MAPSS dataset from: {data_dir}")

    # Prepare data
    train_df, test_df, test_rul_df, scaler, feature_cols = prepare_cmapss_data(
        data_dir,
        'train_FD001.txt',
        'test_FD001.txt',
        'RUL_FD001.txt'
    )

    # Create full training dataset
    full_train_dataset = NASALSTMDataset(train_df, feature_cols, window_size=window_size)

    # Split into train and validation
    val_size = int(len(full_train_dataset) * val_split)
    train_size = len(full_train_dataset) - val_size
    generator = torch.Generator().manual_seed(seed)
    train_dataset, val_dataset = torch.utils.data.random_split(
        full_train_dataset, [train_size, val_size], generator=generator
    )

    # Create test dataset
    test_dataset = NASALSTMDataset(test_df, feature_cols, window_size=window_size,
                                   is_test=True, test_rul_df=test_rul_df)

    # Create data loaders
    num_workers = min(4, os.cpu_count() or 2)
    pin_memory = True if DEVICE.type == 'cuda' else False

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                              num_workers=num_workers, pin_memory=pin_memory)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                            num_workers=num_workers, pin_memory=pin_memory)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                             num_workers=num_workers, pin_memory=pin_memory)

    print(f"DataLoaders created - Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")
    print(f"Input shape: ({window_size}, {len(feature_cols)}) (sequence_length, num_features)")

    return train_loader, val_loader, test_loader, len(feature_cols)


def get_lstm_model(input_size, hidden_size=100, num_layers=2, fc_hidden_sizes=[64, 32], dropout_rate=0.2):
    """Get LSTM model for NASA dataset"""
    model = NASALSTM(input_size, hidden_size, num_layers, fc_hidden_sizes, dropout_rate)
    print(f"✅ Created LSTM with architecture:")
    print(f"   LSTM: input_size={input_size}, hidden_size={hidden_size}, num_layers={num_layers}")
    print(f"   FC: {hidden_size} -> {' -> '.join(map(str, fc_hidden_sizes))} -> 1")
    return model


def get_ignored_layers(model):
    """Get layers to ignore during pruning (typically final layer)"""
    ignored_layers = []

    # Ignore the LSTM layer since torch_pruning only supports single-layer LSTMs
    ignored_layers.append(model.lstm)

    # Get the last linear layer in the fc sequential model
    for module in model.fc:
        if isinstance(module, nn.Linear):
            last_linear = module
    ignored_layers.append(last_linear)

    return ignored_layers


def calculate_macs_params(model, example_input):
    """Calculate MACs and parameters using torch_pruning"""
    model.eval()
    target_device = example_input.device
    model_on_device = model.to(target_device)

    with torch.no_grad():
        macs, params = tp.utils.count_ops_and_params(model_on_device, example_input)

    return macs, params


def save_model(model, save_path, example_input_cpu=None):
    """Save model state dict and optionally ONNX"""
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    torch.save(model.state_dict(), save_path)
    print(f"✅ Model saved to {save_path}")

    if example_input_cpu is not None:
        onnx_path = save_path.replace('.pth', '.onnx')
        try:
            model_cpu = model.to('cpu')
            torch.onnx.export(
                model_cpu, example_input_cpu, onnx_path,
                export_params=True, opset_version=13,
                input_names=['input'], output_names=['output'],
                dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
            )
            print(f"✅ ONNX model saved to {onnx_path}")
        except Exception as e:
            print(f"Warning: ONNX export failed: {e}")


def evaluate_model(model, data_loader, example_input, criterion, device):
    """Evaluate model and return comprehensive metrics"""
    model.eval()
    model.to(device)

    # Calculate efficiency metrics
    macs, params = calculate_macs_params(model, example_input.to(device))
    model_size_mb = params * 4 / (1024 * 1024)  # Assuming float32

    # Calculate MSE and MAE
    total_loss = 0.0
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)

            total_loss += loss.item() * data.size(0)
            all_predictions.extend(output.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)

    mse = np.mean((all_predictions - all_targets) ** 2)
    mae = np.mean(np.abs(all_predictions - all_targets))

    return {
        'mse': mse,
        'mae': mae,
        'loss': total_loss / len(data_loader.dataset),
        'macs': macs,
        'params': params,
        'size_mb': model_size_mb
    }


def prune_model(model, strategy_config, sparsity_ratio, example_input, ignored_layers=None):
    """Apply structured pruning to model"""
    if sparsity_ratio == 0.0:
        print("No pruning needed (sparsity = 0.0)")
        return model

    model.eval()
    pruned_model = copy.deepcopy(model)
    pruned_model.to(example_input.device)

    # Calculate initial MACs
    initial_macs, _ = calculate_macs_params(pruned_model, example_input)
    print(f"Initial MACs: {initial_macs / 1e6:.2f}M")

    ignored_layers = ignored_layers or []

    # Create pruner based on strategy
    # Only prune Linear layers, not LSTM
    pruner = strategy_config['pruner'](
        pruned_model,
        example_input,
        importance=strategy_config['importance'],
        iterative_steps=5,  # Use 5 iterative steps
        ch_sparsity=sparsity_ratio,
        root_module_types=[nn.Linear],  # Only prune Linear layers
        ignored_layers=ignored_layers
    )

    print(f"Applying {strategy_config['importance'].__class__.__name__} pruning at {sparsity_ratio:.1%} sparsity...")
    print("Note: Only pruning FC layers, LSTM layers are preserved")

    # Apply pruning
    pruner.step()

    # Calculate final MACs
    final_macs, _ = calculate_macs_params(pruned_model, example_input)
    reduction = (initial_macs - final_macs) / initial_macs * 100 if initial_macs > 0 else 0
    print(f"Final MACs: {final_macs / 1e6:.2f}M (Reduction: {reduction:.1f}%)")

    return pruned_model


def train_model(model, train_loader, criterion, optimizer, device, num_epochs,
                val_loader=None, patience=7, log_prefix=""):
    """Train model with early stopping"""
    model.to(device)

    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None

    history = {
        'train_loss': [],
        'train_mse': [],
        'val_loss': [],
        'val_mse': []
    }

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_predictions = []
        train_targets = []

        for data, target in train_loader:
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_predictions.extend(output.detach().cpu().numpy())
            train_targets.extend(target.cpu().numpy())

        avg_train_loss = train_loss / len(train_loader)
        train_mse = np.mean((np.array(train_predictions) - np.array(train_targets)) ** 2)

        history['train_loss'].append(avg_train_loss)
        history['train_mse'].append(train_mse)

        log_msg = f"Epoch {epoch + 1}/{num_epochs} ({log_prefix}): Train Loss: {avg_train_loss:.4f}, Train MSE: {train_mse:.2f}"

        # Validation phase
        if val_loader:
            model.eval()
            val_loss = 0.0
            val_predictions = []
            val_targets = []

            with torch.no_grad():
                for data, target in val_loader:
                    data, target = data.to(device), target.to(device)
                    output = model(data)
                    loss = criterion(output, target)

                    val_loss += loss.item()
                    val_predictions.extend(output.cpu().numpy())
                    val_targets.extend(target.cpu().numpy())

            avg_val_loss = val_loss / len(val_loader)
            val_mse = np.mean((np.array(val_predictions) - np.array(val_targets)) ** 2)

            history['val_loss'].append(avg_val_loss)
            history['val_mse'].append(val_mse)

            log_msg += f", Val Loss: {avg_val_loss:.4f}, Val MSE: {val_mse:.2f}"

            # Early stopping check
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                epochs_no_improve = 0
                best_model_state = copy.deepcopy(model.state_dict())
                log_msg += " (Best)"
            else:
                epochs_no_improve += 1

            if epochs_no_improve >= patience:
                print(f"{log_msg}")
                print(f"Early stopping triggered after {epoch + 1} epochs")
                break
        else:
            history['val_loss'].append(None)
            history['val_mse'].append(None)

        print(log_msg)

    # Load best model state if available
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print("Loaded best model state")

    return model, history


def save_results_to_files(all_results, output_dir):
    """Save experimental results to JSON and CSV files"""
    os.makedirs(output_dir, exist_ok=True)

    # Save complete results as JSON
    results_json_path = os.path.join(output_dir, 'complete_results.json')
    with open(results_json_path, 'w') as f:
        json.dump(all_results, f, indent=2, default=str)
    print(f"✅ Complete results saved to {results_json_path}")

    # Create summary DataFrame
    summary_data = []
    for strategy, strategy_results in all_results.items():
        for sparsity, metrics in strategy_results.items():
            row = {
                'strategy': strategy,
                'sparsity_ratio': sparsity,
                'mse': metrics['mse'],
                'mae': metrics['mae'],
                'loss': metrics['loss'],
                'macs_millions': metrics['macs'] / 1e6,
                'params_millions': metrics['params'] / 1e6,
                'size_mb': metrics['size_mb']
            }
            summary_data.append(row)

    # Save summary as CSV
    summary_df = pd.DataFrame(summary_data)
    summary_csv_path = os.path.join(output_dir, 'summary_results.csv')
    summary_df.to_csv(summary_csv_path, index=False)
    print(f"✅ Summary results saved to {summary_csv_path}")

    return summary_df


def create_results_plots(summary_df, output_dir):
    """Create visualization plots for regression metrics"""
    os.makedirs(output_dir, exist_ok=True)

    strategies = summary_df['strategy'].unique()
    sparsity_levels = sorted(summary_df['sparsity_ratio'].unique())

    # Plot 1: MSE vs Sparsity
    plt.figure(figsize=(10, 6))
    for strategy in strategies:
        strategy_data = summary_df[summary_df['strategy'] == strategy].sort_values('sparsity_ratio')
        plt.plot(strategy_data['sparsity_ratio'] * 100, strategy_data['mse'],
                 'o-', linewidth=2, markersize=8, label=strategy)

    plt.xlabel('Sparsity (%)', fontsize=12)
    plt.ylabel('MSE', fontsize=12)
    plt.title('NASA LSTM: MSE vs Sparsity (FC Layers Only)', fontsize=14, fontweight='bold')
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.gca().invert_yaxis()  # Lower MSE is better
    plt.tight_layout()

    plot_path = os.path.join(output_dir, 'mse_vs_sparsity.png')
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✅ MSE plot saved to {plot_path}")

    # Plot 2: Efficiency frontier (MSE vs MACs)
    plt.figure(figsize=(10, 6))
    for strategy in strategies:
        strategy_data = summary_df[summary_df['strategy'] == strategy].sort_values('sparsity_ratio')
        plt.scatter(strategy_data['macs_millions'], strategy_data['mse'],
                    s=100, label=strategy, alpha=0.8)
        plt.plot(strategy_data['macs_millions'], strategy_data['mse'],
                 '--', alpha=0.6)

    plt.xlabel('MACs (Millions)', fontsize=12)
    plt.ylabel('MSE', fontsize=12)
    plt.title('NASA LSTM: Efficiency Frontier (MSE vs MACs)', fontsize=14, fontweight='bold')
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.gca().invert_yaxis()  # Lower MSE is better
    plt.tight_layout()

    plot_path = os.path.join(output_dir, 'efficiency_frontier.png')
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✅ Efficiency frontier plot saved to {plot_path}")


def print_results_table(summary_df):
    """Print formatted results table"""
    print("\n" + "=" * 80)
    print("EXPERIMENTAL RESULTS SUMMARY")
    print("=" * 80)

    # Baseline results
    baseline_results = summary_df[summary_df['sparsity_ratio'] == 0.0].iloc[0]
    print(f"\nBaseline Performance:")
    print(f"  MSE: {baseline_results['mse']:.2f}")
    print(f"  MAE: {baseline_results['mae']:.2f}")
    print(f"  MACs: {baseline_results['macs_millions']:.2f}M")
    print(f"  Parameters: {baseline_results['params_millions']:.2f}M")
    print(f"  Model Size: {baseline_results['size_mb']:.2f}MB")

    # Strategy comparison at 50% sparsity
    print(f"\nStrategy Comparison at 50% Sparsity (FC Layers Only):")
    sparsity_50_data = summary_df[summary_df['sparsity_ratio'] == 0.5]
    for _, row in sparsity_50_data.iterrows():
        degradation = row['mse'] - baseline_results['mse']
        relative_increase = (degradation / baseline_results['mse']) * 100
        print(
            f"  {row['strategy']:>12}: MSE={row['mse']:>6.2f} ({degradation:>+5.2f}, {relative_increase:>+5.1f}% increase)")

    # Complete results table
    print(f"\nComplete Results Table:")
    print("-" * 90)
    print(f"{'Strategy':<12} {'Sparsity':<8} {'MSE':<8} {'MAE':<8} {'MACs(M)':<8} {'Params(M)':<9} {'Size(MB)':<8}")
    print("-" * 90)

    for _, row in summary_df.sort_values(['strategy', 'sparsity_ratio']).iterrows():
        print(f"{row['strategy']:<12} {row['sparsity_ratio'] * 100:>6.0f}% "
              f"{row['mse']:>7.2f} {row['mae']:>7.2f} {row['macs_millions']:>7.2f} "
              f"{row['params_millions']:>8.2f} {row['size_mb']:>7.2f}")

    print("\nNote: Sparsity is applied only to FC layers. LSTM layers remain unchanged.")


def main():
    """Main experimental workflow"""
    print("Starting NASA LSTM Pruning Experiments")
    print("=" * 60)

    # Configuration
    config = {
        'strategies': {
            'MagnitudeL2': {
                'pruner': tp.pruner.MagnitudePruner,
                'importance': tp.importance.MagnitudeImportance(p=2)
            },
            'Random': {
                'pruner': tp.pruner.MagnitudePruner,
                'importance': tp.importance.RandomImportance()
            },
            # Note: BNScale is not applicable to LSTM as it doesn't have BatchNorm layers
        },
        'pruning_ratios': [0.0, 0.2, 0.5, 0.7],
        'hidden_size': 100,
        'num_layers': 2,
        'fc_hidden_sizes': [64, 32],
        'dropout_rate': 0.2,
        'window_size': 50,
        'batch_size': 128,
        'learning_rate': 0.0001,
        'epochs': 1000,  # More epochs for time series
        'patience': 20,
        'output_dir': './results_lstm_nasa',
        'models_dir': './models_lstm_nasa',
        'data_dir': './data/CMaps'
    }

    # Create output directories
    os.makedirs(config['output_dir'], exist_ok=True)
    os.makedirs(config['models_dir'], exist_ok=True)

    # Load data
    print("Loading NASA C-MAPSS dataset...")
    train_loader, val_loader, test_loader, input_size = get_data_loaders(
        data_dir=config['data_dir'],
        batch_size=config['batch_size'],
        window_size=config['window_size']
    )

    # Prepare inputs and criterion
    example_input_cpu = torch.randn(1, config['window_size'], input_size)
    example_input_device = example_input_cpu.to(DEVICE)
    criterion = nn.MSELoss()  # Regression loss

    # Get baseline model and train it
    print("\nCreating and training baseline model...")
    model = get_lstm_model(
        input_size,
        config['hidden_size'],
        config['num_layers'],
        config['fc_hidden_sizes'],
        config['dropout_rate']
    )
    model.to(DEVICE)

    # Train baseline model
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
    trained_model, training_history = train_model(
        model, train_loader, criterion, optimizer, DEVICE,
        config['epochs'], val_loader, config['patience'], "Baseline Training"
    )

    # Save baseline model
    baseline_model_path = os.path.join(config['models_dir'], 'baseline_model.pth')
    save_model(trained_model, baseline_model_path, example_input_cpu)

    # Evaluate baseline
    print("\nEvaluating baseline model...")
    baseline_metrics = evaluate_model(trained_model, test_loader, example_input_device, criterion, DEVICE)
    print(f"Baseline Results: MSE={baseline_metrics['mse']:.2f}, "
          f"MAE={baseline_metrics['mae']:.2f}, "
          f"MACs={baseline_metrics['macs'] / 1e6:.2f}M, "
          f"Params={baseline_metrics['params'] / 1e6:.2f}M")

    # Initialize results storage
    all_results = {}
    for strategy_name in config['strategies'].keys():
        all_results[strategy_name] = {0.0: baseline_metrics}

    # Get ignored layers
    ignored_layers = get_ignored_layers(trained_model)

    # Run pruning experiments
    print("\nStarting pruning experiments...")
    print("Note: Due to torch_pruning limitations, only FC layers will be pruned (LSTM layers preserved)")

    for strategy_name, strategy_config in config['strategies'].items():
        print(f"\n--- Strategy: {strategy_name} ---")

        for sparsity_ratio in config['pruning_ratios']:
            if sparsity_ratio == 0.0:
                continue  # Skip baseline (already done)

            print(f"\nProcessing {strategy_name} at {sparsity_ratio:.1%} sparsity...")

            # Load fresh copy of trained baseline
            model_copy = get_lstm_model(
                input_size,
                config['hidden_size'],
                config['num_layers'],
                config['fc_hidden_sizes'],
                config['dropout_rate']
            )
            model_copy.load_state_dict(torch.load(baseline_model_path, map_location=DEVICE))
            model_copy.to(DEVICE)

            # Apply pruning
            pruned_model = prune_model(
                model_copy, strategy_config, sparsity_ratio,
                example_input_device, ignored_layers
            )

            # Fine-tune pruned model
            print("Fine-tuning pruned model...")
            optimizer_ft = optim.Adam(pruned_model.parameters(), lr=config['learning_rate'])
            fine_tuned_model, ft_history = train_model(
                pruned_model, train_loader, criterion, optimizer_ft, DEVICE,
                config['epochs'], val_loader, config['patience'],
                f"{strategy_name}-{sparsity_ratio:.1%}"
            )

            # Evaluate fine-tuned model
            final_metrics = evaluate_model(fine_tuned_model, test_loader, example_input_device, criterion, DEVICE)
            all_results[strategy_name][sparsity_ratio] = final_metrics

            print(f"Results: MSE={final_metrics['mse']:.2f}, "
                  f"MAE={final_metrics['mae']:.2f}, "
                  f"MACs={final_metrics['macs'] / 1e6:.2f}M")

            # Save fine-tuned model
            model_filename = f"{strategy_name.lower()}_sparsity_{sparsity_ratio:.1f}.pth"
            model_path = os.path.join(config['models_dir'], model_filename)
            save_model(fine_tuned_model, model_path, example_input_cpu)

    # Save and analyze results
    print("\nSaving results...")
    summary_df = save_results_to_files(all_results, config['output_dir'])

    # Create plots
    print("Creating plots...")
    create_results_plots(summary_df, config['output_dir'])

    # Print summary
    print_results_table(summary_df)

    print(f"\n🎉 All experiments completed!")
    print(f"📁 Results saved to: {os.path.abspath(config['output_dir'])}")
    print(f"📁 Models saved to: {os.path.abspath(config['models_dir'])}")


if __name__ == "__main__":
    main()

Using device: cuda
Starting NASA LSTM Pruning Experiments
Loading NASA C-MAPSS dataset...
Loading NASA C-MAPSS dataset from: ./data/CMaps
--- Preparing Training Data ---

--- Preparing Test Data ---
Columns with std < 0.02 (potential removal): ['op_setting_1', 'op_setting_2', 'op_setting_3', 'sensor_1', 'sensor_5', 'sensor_6', 'sensor_10', 'sensor_16', 'sensor_18', 'sensor_19']

Using Features: ['sensor_2', 'sensor_3', 'sensor_4', 'sensor_7', 'sensor_8', 'sensor_9', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15', 'sensor_17', 'sensor_20', 'sensor_21']

--- Normalizing Data ---
DataLoaders created - Train: 12585, Val: 3146, Test: 100
Input shape: (50, 14) (sequence_length, num_features)

Creating and training baseline model...
✅ Created LSTM with architecture:
   LSTM: input_size=14, hidden_size=100, num_layers=2
   FC: 100 -> 64 -> 32 -> 1
Epoch 1/1000 (Baseline Training): Train Loss: 7318.2809, Train MSE: 7316.14, Val Loss: 7437.0442, Val MSE: 7429.02 (Best)
Epoch 2/1

/home/muis/.virtualenvs/master-thesis/lib/python3.12/site-packages/torch/onnx/symbolic_opset9.py:4277: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
/home/muis/.virtualenvs/master-thesis/lib/python3.12/site-packages/torch/nn/modules/rnn.py:1124: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1412.)
  result = _VF.lstm(


Baseline Results: MSE=205.94, MAE=10.50, MACs=90.45M, Params=0.14M

Starting pruning experiments...
Note: Due to torch_pruning limitations, only FC layers will be pruned (LSTM layers preserved)

--- Strategy: MagnitudeL2 ---

Processing MagnitudeL2 at 20.0% sparsity...
✅ Created LSTM with architecture:
   LSTM: input_size=14, hidden_size=100, num_layers=2
   FC: 100 -> 64 -> 32 -> 1
Initial MACs: 90.45M
Applying MagnitudeImportance pruning at 20.0% sparsity...
Note: Only pruning FC layers, LSTM layers are preserved
Final MACs: 90.45M (Reduction: 0.0%)
Fine-tuning pruned model...


/home/muis/.virtualenvs/master-thesis/lib/python3.12/site-packages/torch_pruning/pruner/algorithms/base_pruner.py:87: UserWarning: ch_sparsity is deprecated in v1.3.0. Please use pruning_ratio.
  warnings.warn(


Epoch 1/1000 (MagnitudeL2-20.0%): Train Loss: 320.8533, Train MSE: 321.48, Val Loss: 127.4979, Val MSE: 127.70 (Best)
Epoch 2/1000 (MagnitudeL2-20.0%): Train Loss: 326.8225, Train MSE: 327.41, Val Loss: 134.5940, Val MSE: 134.82
Epoch 3/1000 (MagnitudeL2-20.0%): Train Loss: 327.7182, Train MSE: 328.15, Val Loss: 165.8171, Val MSE: 165.68
Epoch 4/1000 (MagnitudeL2-20.0%): Train Loss: 318.7881, Train MSE: 318.59, Val Loss: 147.8816, Val MSE: 147.97
Epoch 5/1000 (MagnitudeL2-20.0%): Train Loss: 313.9571, Train MSE: 314.50, Val Loss: 138.1769, Val MSE: 137.98
Epoch 6/1000 (MagnitudeL2-20.0%): Train Loss: 319.8437, Train MSE: 319.51, Val Loss: 136.4381, Val MSE: 136.70
Epoch 7/1000 (MagnitudeL2-20.0%): Train Loss: 329.2139, Train MSE: 329.45, Val Loss: 131.1303, Val MSE: 131.07
Epoch 8/1000 (MagnitudeL2-20.0%): Train Loss: 317.8225, Train MSE: 317.75, Val Loss: 137.5211, Val MSE: 137.27
Epoch 9/1000 (MagnitudeL2-20.0%): Train Loss: 308.6468, Train MSE: 307.95, Val Loss: 129.9778, Val MSE: 1